In [ ]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [ ]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

# Preparing Data

* music21 라이브러리 함수 이해

 * music21.converter : 다양한 음악 파일 포맷으로부터 음악을 로드하는 툴을 가지고 있습니다.
 * music21.converter.parse() : 파일의 아이템을 파싱하고 스트림에 넣어줍니다.
 * instrument.partitionByInstrument() : 단일 스트림, 스코어, 멀티 파트 구조인 경우에 각각의 악기별로 파티션을 나누고 다른 파트들을 하나로 합쳐줍니다.
 * Stream.recurse : 스트림안에 존재하는 Music21 객체가 가지고 있는 값들의 리스트를 반복(iterate)할 수 있는 iterator 를 리턴해줍니다.

In [ ]:
from music21 import converter, instrument, note, chord
import glob  # python을 사용하여 디렉토리 안의 특정 확장자 또는 모든 파일을 찾을 수 있습니다.


def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)    # coverter : 다양한 음악파일 포맷으로 음악을 로드,   .parse : file의 item을 parsing하고, stream에 넣어줌
        
                                        # 각 파일을 Music21 스트림 개체로 로드! 스트림 개체를 사용하면 파일에 있는 모든 노트와 코드(chord) 목록이 나옵니다.
                                        # 파싱((syntactic) parsing)은 일련의 문자열을 의미있는 토큰(token)으로 분해하고 이들로 이루어진 파스 트리(parse tree)를 만드는 과정을 말한다.
        print("Parsing %s" % file)

        notes_to_parse = None

        # Music21 라이브러리
        
        try: # file has instrument parts  
            s2 = instrument.partitionByInstrument(midi)    #  단일 스트림, 스코어, 멀티 파트 구조인 경우에 각각의 악기별로 파티션을 나누고 다른 파트들을 하나로 합쳐줍니다.
            notes_to_parse = s2.parts[0].recurse()    
        except: # file has notes in a flat structure   
            notes_to_parse = midi.flat.notes

            
        for element in notes_to_parse:
            if isinstance(element, note.Note):   # instance가 특정 클래스/데이터 타입과 일치할 경우에는 True를 아닌 경우에는 False를 return해줍니다. 
                notes.append(str(element.pitch))  # 모든 note object의 pitch(계이름)을 string notaion으로 추가
            elif isinstance(element, chord.Chord):   
                notes.append('.'.join(str(n) for n in element.normalOrder))   # 우리는 각 음을 점으로 나누어서 현 안에 있는 모든 음들의 id를 하나의 문자열로 인코딩함으로써 모든 화음을 추가
                # 이러한 인코딩을 통해 네트워크에 의해 생성된 출력을 올바른 노트와 코드(chord)로 쉽게 디코딩할 수 있습니다.
                # 이제 모든 노트와 코드(chord)를 순차적 목록에 넣었다   ------> 네트워크의 입력으로 사용될 시퀀스를 만들 수 있습니다.
                
                
                                                   # pickle 모듈을 활용하여 데이터 입력 및 로드
    with open('data/notes', 'wb') as filepath:     #  pickle로 데이터를 저장하거나 불러올때는 파일을 바이트형식으로 읽거나 써야한다. (wb, rb)
        pickle.dump(notes, filepath)

    return notes

# Create the sequences that will serve as the input of our network
* 네트워크의 입력으로 사용될 시퀀스를 만들기

In [ ]:

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100     # put the length of each sequence to be 100 notes/chords
                              # 네트워크에서 다음 노트를 예측하기 위해서 도움이 되는 이전 100개의 노트를 사용한다

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))     # 모든 계이름의 이름을 pitchnames 변수에 저장
                                                         # set 으로 중복을 피하고, sorted 함수로 정렬함

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))    # 각 계이름(문자열)을 숫자로 바꾸는 dictionary(사전)을 만든다 -> 숫자기반데이터를 신경망이 더 잘 학습

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs      # 입력 시퀀스를 만든다
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers                # 데이터 입력 형태를 LSTM 레이어에 알맞게 변경함
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input   # 입력값을 normalizing(정규화)
    network_input = network_input / float(n_vocab)
    network_output = np_utils.to_categorical(network_output)
    
    #네트워크 데이터 준비의 마지막 단계는 입력을 정규화하고 출력을 원-핫 인코딩하는 것입니다.


    return (network_input, network_output)


# Model (모델) architecture 설계

* 모델은 4가지 유형의 레이어 사용

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),     #  input_shape 매개변수를 사용하는 목적은 모델을 학습하기 위해서 쓸 데이터의 형태를 네트워크에 알리는 것입니다.
        recurrent_dropout=0.3,
        return_sequences=True                     # return_sequences: 불리언. 아웃풋 시퀀스의 마지막 아웃풋을 반환할지, 혹은 시퀀스 전체를 반환할지 여부.
    ))                                            # return_sequences가 True이므로 첫번째 출력값은 모든 시점의 은닉 상태가 출력됩니다.
                                                  # recurrent_dropout: 0과 1사이 부동소수점. 순환 상태의 선형적 변형을 실행하는데 드롭시킬(고려하지 않을) 유닛의 비율.
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))      # LSTM 레이어 는 어떤 시퀀스를 입력으로 넣었을 때 출력으로 또 다른 시퀀스 또는 행렬을 주는 순환신경망입니다
    model.add(BatchNorm())    # 케라스에서 배치 정규화는 하나의 레이어(BatchNormalization())처럼 작동하며, 보통 Dense 혹은 Convolution 레이어와 활성함수(Activation) 레이어 사이에 들어간다.
    model.add(Dropout(0.3))   # Dropout 레이어 는무작위로 학습을 쓸 뉴런을 정해서 학습을 진행하는 것입니다. mini-batch 마다 랜덤하게 되는 뉴런이 달라지기 때문에 다양한 모델을 쓰는듯한 효과를 냅니다.
    model.add(Dense(256))    # Dense 레이어 또는 Fully connected 레이어는 이전 레이어의 모든 뉴런과 결합된 형태의 레이어
    model.add(Activation('relu'))  # Activation 레이어는 신경망이 노드의 출력을 계산하는 데 사용할 활성화 기능을 결정합니다 # (relu) rectifier 함수, 은닉층에 주로 쓰입니다.
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))  # ‘softmax’ : 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 쓰입니다.
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')   # 반복에 대한 손실을 계산하기 위해, 우리는 categorical cross entropy(범주형 크로스 엔트로피)를 사용
                                                                          # 범주형 크로스 엔트로피를 사용하는 이유는 각 출력은 단 한 개의 클래스에만 속해야하고 우리는 두 개 이상의 클래스를 가지고 있기 때문
                                                                          # 네트워크를 최적화하기 위해 RMSprop optimizer를 사용할 것입니다. 
                                                                          # RMSprop optimizer는 일반적으로 순환신경망에 매우 적합합니다. 
    
    
    
    
    # 각 LSTM, Dense 및 Activation 레이어에 대해 첫 번째 매개변수는 레이어가 가져야 하는 노드 수입니다. 
    # Dropout 레이어의 경우 첫 번째 매개변수는 교육 중에 삭제해야 하는 입력 단위의 비율*입니다. 전체의 노드(위에서 설명한 무작위로 정할 뉴런)를 결정하는 비율입니다.

    # 마지막 레이어은 항상 시스템의 서로 다른 출력 수와 동일한 수의 노드를 포함해야 합니다. 이는 네트워크 출력이 우리 클래스와 직접 연결되도록 보장합니다.
    
    return model


# Train

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    # 우리가 열심히 학습한 것을 잃지 않고도 언제든 훈련을 중단할 수 있도록 우리는 모델 체크포인트를 사용할 것입니다. 
    # 모델 체크포인트는 매 에폭마다 네트워크 노드의 가중치를 파일에 저장할 수 있는 방법을 제공합니다.
    # 이것은 우리가 가중치를 잃어 버릴 걱정 없이 손실 값에 만족하면 신경망을 작동시키는 것을 멈출 수 있게 해줍니다.
    
    
    callbacks_list = [checkpoint]
    
    

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)
    
   # Keras의 model.fit() 함수는 신경망을 학습하는 데 사용됩니다. model.fit() 함수의 첫 번째 파라미터는 앞에서 준비한 입력 시퀀스 목록이고 두 번째 파라미터는 각 출력의 목록입니다. 
   # 64개의 샘플을 포함하는 네트워크를 통해 전파되는 각 배치에 대해 200 에폭(반복, epoch) 동안 네트워크를 학습할 것입니다.

if __name__ == '__main__':
    train_network()
